# rdd test

This is a text cell. Start editing!

In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import com.johnsnowlabs.ml.tensorflow.TensorflowBert
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector

import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}

import org.apache.spark.ml.linalg.Vectors


val dataDir = sys.env("HOME") + "/recsys2020"
val dsName = "training1m"

In [2]:
val schema = new StructType()
    .add("text_tokens", StringType, true)
    .add("hashtags", StringType, true)
    .add("tweet_id", StringType, true)
    .add("present_media", StringType, true)
    .add("present_links", StringType, true)
    .add("present_domains", StringType, true)
    .add("tweet_type", StringType, true)
    .add("language", StringType, true)
    .add("tweet_timestamp", IntegerType, true)
    .add("author_id", StringType, true)
    .add("author_follower_count", IntegerType, true)
    .add("author_following_count", IntegerType, true)
    .add("author_is_verified", BooleanType, true)
    .add("author_account_creation", IntegerType, true)
    .add("user_id", StringType, true)
    .add("user_follower_count", IntegerType, true)
    .add("user_following_count", IntegerType, true)
    .add("user_is_verified", BooleanType, true)
    .add("user_account_creation", IntegerType, true)
    .add("follows", BooleanType, true)
    .add("reply_timestamp", IntegerType, true)
    .add("retweet_timestamp", IntegerType, true)
    .add("retweet_with_comment_timestamp", IntegerType, true)
    .add("like_timestamp", IntegerType, true)

val df = spark.read.format("csv")
        .option("delimiter", "\u0001")
        .schema(schema)
        .load(dataDir + s"/${dsName}.tsv")
        .limit(1000)

In [3]:
df.show

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+--------------------+---------------+--------------------+---------------------+----------------------+------------------+-----------------------+--------------------+-------------------+--------------------+----------------+---------------------+-------+---------------+-----------------+------------------------------+--------------+
|         text_tokens|            hashtags|            tweet_id|present_media|       present_links|     present_domains|tweet_type|            language|tweet_timestamp|           author_id|author_follower_count|author_following_count|author_is_verified|author_account_creation|             user_id|user_follower_count|user_following_count|user_is_verified|user_account_creation|follows|reply_timestamp|retweet_timestamp|retweet_with_comment_timestamp|like_timestamp|
+--------------------+--------------------+--------------------+--

In [4]:
val bert = BertEmbeddings.pretrained(name="bert_multi_cased", lang="xx")

bert_multi_cased download started this may take some time.
Approximate size to download 638.7 MB
Download done! Loading the resource.


In [5]:
val maxSentenceLength = 100

df.rdd.mapPartitions(partition =>
    partition.grouped(100).map { group => 
        val sentenceGroup = for (record <- group) yield { 
            val tokens = record.getAs[String]("text_tokens").split("\t").map(_.toInt).toArray
            val numTokens = tokens.length
            tokens ++ Array.fill(maxSentenceLength - numTokens)(0)
        }
        val input = sentenceGroup.toSeq

        // use transformation on aggregate
        // val embeddings_sequence = bert.getModelIfNotSet.tag(input, "module/bert/encoder/Reshape_1:0", bert.getMaxSentenceLength)

        // write back
        // embeddings_sequence.map { embeddings =>
        //     val sumEmbeddings = embeddings.head.transpose.map(_.sum)
        //     sumEmbeddings.map(x => (x * 1.0 / embeddings.length).toFloat)
        // }
        input
    }
).flatMap(x => x)
.toDF
// .count
.show(false)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                             